# 1. Scraping a wikipedia page to get information about neighborhoods in Toronto and create a table. 

In [4]:
!pip install bs4
!pip install html5lib
!pip install folium
!pip install kmeans

In [22]:
from bs4 import BeautifulSoup
import requests  
import pandas as pd
import folium
import requests 
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [23]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [24]:
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

In [25]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [26]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [27]:
df.shape

(103, 3)

# 2. Getting the latitude and the longitude coordinates of each neighborhood. 

In [28]:
geo=pd.read_csv('Geospatial_Coordinates.csv')
geo.shape

(103, 3)

In [29]:
df_sorted=df.sort_values(by=['PostalCode'])
df_r=df_sorted.reset_index(drop=True)


In [30]:
df2=pd.concat([df_r,geo],axis=1)

In [31]:
df2.drop(['Postal Code'],axis=1,inplace=True)

In [32]:
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]))

Toronto has 15 boroughs and 103 neighborhoods.


# 3. Explore and cluster the neighborhoods in Toronto

### Create a map of Toronto and its neighborhoods using latitude and longitude values

In [34]:
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Using the Foursquare API to explore the neighborhoods and segment them

In [35]:
CLIENT_ID = 'EA24PWBDYH1ZZQBWDRQEOGZVDDUEAYKHZ3OOTVGFP3CLRJ0T'
CLIENT_SECRET = 'FROZNBYP2JUVWRWEB2B0RLIL4UDMX30TEJWL25PBF421VTYC' 
VERSION = '20211005'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EA24PWBDYH1ZZQBWDRQEOGZVDDUEAYKHZ3OOTVGFP3CLRJ0T
CLIENT_SECRET:FROZNBYP2JUVWRWEB2B0RLIL4UDMX30TEJWL25PBF421VTYC


In [36]:
neighborhood_latitude = df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8066863, -79.1943534.


In [47]:
LIMIT=1000
radius=5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EA24PWBDYH1ZZQBWDRQEOGZVDDUEAYKHZ3OOTVGFP3CLRJ0T&client_secret=FROZNBYP2JUVWRWEB2B0RLIL4UDMX30TEJWL25PBF421VTYC&v=20211005&ll=43.8066863,-79.1943534&radius=5000&limit=1000'

In [51]:
 def getNearbyVenues(names, latitudes, longitudes, radius=5000):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:

toronto_venues = getNearbyVenues(names=df2['Neighborhood'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,"Malvern, Rouge",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,"Malvern, Rouge",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
3,"Malvern, Rouge",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
4,"Malvern, Rouge",43.806686,-79.194353,Morningside Park,43.786546,-79.205322,Park


In [53]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))

10258 venues were returned by Foursquare.


In [55]:
print('There are {} uniques venues categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 244 uniques venues categories.


In [56]:
print(toronto_venues.shape)
toronto_venues.head()

(10258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,"Malvern, Rouge",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,"Malvern, Rouge",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
3,"Malvern, Rouge",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
4,"Malvern, Rouge",43.806686,-79.194353,Morningside Park,43.786546,-79.205322,Park


### Grouping rows by neighborhood by taking the mean of the frequency of occurrence of each category

In [58]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']])
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Venue Category_Afghan Restaurant,Venue Category_African Restaurant,Venue Category_Airport,Venue Category_Airport Lounge,Venue Category_American Restaurant,Venue Category_Amphitheater,Venue Category_Art Gallery,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,...,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Xinjiang Restaurant,Venue Category_Yoga Studio,Venue Category_Zoo,Venue Category_Zoo Exhibit
0,Agincourt,0.00,0.0,0.00,0.0,0.01,0.0,0.0,0.01,0.01,...,0.0,0.02,0.00,0.0,0.000000,0.00,0.010000,0.00,0.0,0.0
1,"Alderwood, Long Branch",0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.01,0.00,...,0.0,0.01,0.01,0.0,0.000000,0.01,0.000000,0.02,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.00,0.0,0.01,0.0,0.00,0.0,0.0,0.00,0.01,...,0.0,0.00,0.01,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
3,Bayview Village,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.0,0.01,0.00,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.01,...,0.0,0.00,0.01,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Willowdale, Newtonbrook",0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.0,0.01,0.00,0.0,0.010000,0.00,0.000000,0.00,0.0,0.0
99,Woburn,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,...,0.0,0.00,0.00,0.0,0.010309,0.00,0.010309,0.00,0.0,0.0
100,Woodbine Heights,0.01,0.0,0.00,0.0,0.02,0.0,0.0,0.00,0.01,...,0.0,0.00,0.01,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0
101,York Mills West,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.02,...,0.0,0.00,0.00,0.0,0.000000,0.00,0.000000,0.00,0.0,0.0


### Creating a new dataframe with the top 10 venues for each neighborhood

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Venue Category_Chinese Restaurant,Venue Category_Coffee Shop,Venue Category_Caribbean Restaurant,Venue Category_Sandwich Place,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Noodle House,Venue Category_Park,Venue Category_Sushi Restaurant,Venue Category_Pizza Place
1,"Alderwood, Long Branch",Venue Category_Bakery,Venue Category_Café,Venue Category_Burger Joint,Venue Category_Coffee Shop,Venue Category_Burrito Place,Venue Category_Breakfast Spot,Venue Category_Grocery Store,Venue Category_Park,Venue Category_Seafood Restaurant,Venue Category_Sporting Goods Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Venue Category_Coffee Shop,Venue Category_Grocery Store,Venue Category_Clothing Store,Venue Category_Bakery,Venue Category_Restaurant,Venue Category_Middle Eastern Restaurant,Venue Category_Burger Joint,Venue Category_Bubble Tea Shop,Venue Category_Sushi Restaurant,Venue Category_Greek Restaurant
3,Bayview Village,Venue Category_Coffee Shop,Venue Category_Supermarket,Venue Category_Bakery,Venue Category_Middle Eastern Restaurant,Venue Category_Korean Restaurant,Venue Category_Park,Venue Category_Furniture / Home Store,Venue Category_Japanese Restaurant,Venue Category_Bagel Shop,Venue Category_Grocery Store
4,"Bedford Park, Lawrence Manor East",Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Park,Venue Category_Grocery Store,Venue Category_Bakery,Venue Category_Sushi Restaurant,Venue Category_Thai Restaurant,Venue Category_Supermarket,Venue Category_Burger Joint,Venue Category_Liquor Store


### Cluster Neighborhoods

In [63]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df2

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [65]:
toronto_merged=toronto_merged.astype({"Cluster Labels": int})

In [66]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters